<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Johns Hopkins - Get Covid19 data
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/WSR/John_Hopkins_Active_covid_cases_worldmap.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #johnshopkins #opendata #analytics

**Author:** [Florent Ravenel](https://www.linkedin.com/in/ACoAABCNSioBW3YZHc2lBHVG0E_TXYWitQkmwog/)

## Input

### Import libraries

In [1]:
import pandas as pd
import naas

### Variables

In [2]:
# Input URLs of the raw csv dataset
urls = [
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
]

# Output paths
title = "DB_COVID19_JHU"
output_csv = f"{title}.csv"

## Model

### Get data from JHU

In [3]:
def get_data_url(urls):
    df = pd.DataFrame()
    for url in urls:
        tmp_df = pd.read_csv(url)
        tmp_df["Indicator"] = url.split("/time_series_covid19_")[-1].split("_global.csv")[0].capitalize()
        df = pd.concat([df, tmp_df])
    return df

df_init = get_data_url(urls)
df_init

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/26/22,2/27/22,2/28/22,3/1/22,3/2/22,3/3/22,3/4/22,3/5/22,3/6/22,Indicator
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,173146,173395,173659,173879,174073,174214,174214,174331,174582,Confirmed
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,271141,271527,271563,271702,271825,271825,272030,272030,272210,Confirmed
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,264778,264855,264936,265010,265079,265130,265186,265227,265265,Confirmed
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,37999,37999,37999,38165,38249,38342,38434,38434,38434,Confirmed
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,98701,98701,98741,98746,98746,98746,98796,98796,98806,Confirmed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Recovered
265,NaN,Winter Olympics 2022,39.904200,116.407400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Recovered
266,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Recovered
267,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Recovered


### Get all data from JHU

In [4]:
def get_all_data(df_init):
    df = df_init.copy()
    # Cleaning
    df = df.drop("Province/State", axis=1)
    
    # Melt data
    df = pd.melt(df,
                 id_vars=["Country/Region", "Lat", "Long", "Indicator"],
                 var_name="Date",
                 value_name="Value").fillna(0)
    df["Date"] = pd.to_datetime(df["Date"])
    
    # Calc active cases
    df_active = df.copy()
    df_active.loc[df_active["Indicator"].isin(["Deaths", "Recovered"]), "Value"] = df_active["Value"] * (-1)
    df_active["Indicator"] = "Active cases"
    
    # Concat data
    df = pd.concat([df, df_active])
    
    # Group by country/region
    to_group = ["Country/Region", "Lat", "Long", "Indicator", "Date"]
    df = df.groupby(to_group, as_index=False).agg({"Value": "sum"})
    
    # Cleaning
    df = df.rename(columns={"Country/Region": "COUNTRY"})
    df.columns = df.columns.str.upper()
    return df.reset_index(drop=True)

df_clean = get_all_data(df_init)
df_clean

,COUNTRY,LAT,LONG,INDICATOR,DATE,VALUE
0,Afghanistan,33.939110,67.709953,Active cases,2020-01-22,0
1,Afghanistan,33.939110,67.709953,Active cases,2020-01-23,0
2,Afghanistan,33.939110,67.709953,Active cases,2020-01-24,0
3,Afghanistan,33.939110,67.709953,Active cases,2020-01-25,0
4,Afghanistan,33.939110,67.709953,Active cases,2020-01-26,0
...,...,...,...,...,...,...
867995,Zimbabwe,-19.015438,29.154857,Recovered,2022-03-02,0
867996,Zimbabwe,-19.015438,29.154857,Recovered,2022-03-03,0
867997,Zimbabwe,-19.015438,29.154857,Recovered,2022-03-04,0
867998,Zimbabwe,-19.015438,29.154857,Recovered,2022-03-05,0


## Output

### Save dataframe in csv

In [5]:
df_clean.to_csv(output_csv, index=False)